In [1]:
%load_ext autoreload
%autoreload 2
%cd /calc/baronig/Projects/cLSNN/romain_fork/SE-adlif
from IPython.display import Audio
from omegaconf import DictConfig, omegaconf
import torch
import torchaudio
from datasets.audio_compress import LibriTTS
from datasets.utils.diskcache import DiskCachedDataset
from models.pl_module_compress import MLPSNN
import os
import plotly.express as px
import plotly.graph_objects as go


/home/baronig/Projects/cLSNN/romain_fork/SE-adlif


/home/baronig/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [6]:
@torch.compiler.disable
def create_audio_example(cfg_path, ckpt_path, example_idx,return_spike_proba):
    cfg = omegaconf.OmegaConf.load(cfg_path)
    # cfg.decoder.l_out.cell = 'li'
    # dataset = LibriTTS('/home/romain/datasets/LibriTTS/', sampling_freq=16_000, sample_length=-1, prediction_delay=cfg.dataset.prediction_delay)
    # dataset = LibriTTS(save_to='/calc/baronig/data_sets/LibriTTS/', cache_path='/calc/baronig/data_sets/LibriTTS/', sampling_freq=16_000, sample_length=-1)
    dataset = LibriTTS(save_to='/calc/baronig/data_sets/LibriTTS/', cache_path='/scratch/baronig/cache/librispeech', sampling_freq=16_000, sample_length=-1)
    ckpt = torch.load(ckpt_path, map_location='cuda:0')
    ckpt_state_dict = ckpt['state_dict']
    model = MLPSNN(cfg)
    model.load_state_dict(ckpt_state_dict)
    model.to("cuda:0")
    model.eval()
    source_waveform = []
    prediction_waveform = []
    spike_probs = []
    for idx in example_idx:
        inputs, *rest = dataset[idx]
        source_waveform.append(inputs.cpu().numpy().squeeze())
        with torch.no_grad():
            if return_spike_proba:
                states, pred = model.forward_with_states(inputs.unsqueeze(0))
                # spike_prob = torch.mean(states[1][1]).item().cpu().numpy()
                spike_prob = torch.mean(states[1][1]).item()
                spike_probs.append(spike_prob)
            else:
                pred = model(inputs.to("cuda:0").unsqueeze(0))
            pred = pred[:, cfg.dataset.prediction_delay:]
        prediction_waveform.append(pred.cpu().numpy().squeeze())
    return source_waveform, prediction_waveform, spike_probs

In [7]:
def remove_prefix(text, prefix):
    if text.startswith(prefix):
        return "model." + text[len(prefix) :]
    return text


def repair_checkpoint(path):
    ckpt = torch.load(path)
    in_state_dict = ckpt["state_dict"]
    # in_state_dict = ckpt
    pairings = [
        (src_key, remove_prefix(src_key, "model._orig_mod."))
        for src_key in in_state_dict.keys()
    ]
    if all(src_key == dest_key for src_key, dest_key in pairings):
        return  # Do not write checkpoint if no need to repair!
    out_state_dict = {}
    for src_key, dest_key in pairings:
        print(f"{src_key}  ==>  {dest_key}")
        out_state_dict[dest_key] = in_state_dict[src_key]
    ckpt["state_dict"] = out_state_dict
    torch.save(ckpt, path)

In [10]:

# ckp_path = "/calc/baronig/Projects/sim_results/adlif_rebuttal/compression_task/hydra/2024-12-11/14-42-29/ckpt/last.ckpt"
ckp_path = "/calc/baronig/Projects/sim_results/adlif_rebuttal/compression_task/hydra/2025-01-14/16-18-58/ckpt/last.ckpt"
# repair_checkpoint(ckp_path)
cfg_path = os.path.join(os.path.dirname(ckp_path), "..", ".hydra", "config.yaml")
source_waveform, prediction_waveform, spike_probs = create_audio_example(cfg_path, ckp_path, [1, 200, 1529], False)

directory already exist
Wave files are resampled to 16000Hz
Chunk map loaded from /scratch/baronig/cache/librispeech/debug/16000/-1_map.pkl
Waves files are splited into -1 samples length


/tmp/ipykernel_303659/227777108.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location='cuda:0')


RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.

In [5]:
Audio(prediction_waveform[0], rate=16_000)

In [7]:
fig = px.line(y=source_waveform[0], title="Source waveform")
fig.add_trace(go.Scatter(y=prediction_waveform[0], mode='lines', name='Prediction waveform'))
fig.show()